In [1]:
'''
Optuna: A hyperparameter optimization framework. uses Bayesian optimization to find the best 
        hyperparameters for machine learning models.
Bayesian Optimization: A technique in which a relation is built between hyperparameters and
        model performance using a probablistic model, where new hyperparameters are chosen
        based on past results to maximize performance(finding the maxima of an unknown function).
Key Terms:
Study: An optimization task, which contains multiple trials.
Trial: A single execution of the objective function with a specific set of hyperparameters.
Trail Parameters: These are specific hyperparameter values chosen during a trial.
                 "The combination of multiple trials becomes a study."
Objective Function: A user-defined function that takes hyperparameters as input and returns
        a performance metric to be optimized (ex: In classification the the objective function
        could be cross-entropy loss, which optuna seeks to minimize).
Sampler: A component that suggests hyperparameter values for each trial based on past results.
Pruner: A component that stops unpromising trials early to save computational resources.
        
     
'''

'\nOptuna: A hyperparameter optimization framework. uses Bayesian optimization to find the best \n        hyperparameters for machine learning models.\nBayesian Optimization: A technique in which a relation is built between hyperparameters and\n        model performance using a probablistic model, where new hyperparameters are chosen\n        based on past results to maximize performance(finding the maxima of an unknown function).\nKey Terms:\nStudy: An optimization task, which contains multiple trials.\nTrial: A single execution of the objective function with a specific set of hyperparameters.\nTrail Parameters: These are specific hyperparameter values chosen during a trial.\n                 "The combination of multiple trials becomes a study."\nObjective Function: A user-defined function that takes hyperparameters as input and returns\n        a performance metric to be optimized (ex: In classification the the objective function\n        could be cross-entropy loss, which optuna see

In [2]:
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']
df = pd.read_csv(url,names=columns)
df.head()

c:\Users\ravjot\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
#insulin column has 0 values which is not possible, so we will replace 0 with NaN and then impute it with mean
import numpy as np
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)


In [ ]:
# Check for missing values
print(df.isnull().sum())


Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64


In [6]:
#imputing the missing values with mean
df.fillna(df.mean(), inplace=True)

In [7]:
#check if there are any remaining missing values
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [8]:
#Splitting into features and target
X = df.drop('Outcome', axis=1)
y = df['Outcome']

#Splitting into train and test sets(70-30 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

#Scaling the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [9]:
#checking the shape of train and test sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (537, 8)
X_test shape: (231, 8)


In [10]:
#Building a Obejctive Function for Optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

#Define Obhective Function
def objective(trail):
    #Define hyperparameter and their search space
    n_estimators = trail.suggest_int('n_estimators', 50, 200)
    max_depth = trail.suggest_int('max_depth', 3, 20)
    
    #creating a random forest classifier with the suggested hyperparameters
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )
    
    #performing cross-validation
    score = cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy').mean()
    
    return score


In [11]:
#Creating study and optimizing it
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler()) #Aimming to maximize accuracy
study.optimize(objective, n_trials = 50)#running 50 trials for best hyperparameters


[I 2026-01-28 23:49:33,869] A new study created in memory with name: no-name-14bc57bd-a9ac-4c4d-bcbd-a97442d1ea3b
[I 2026-01-28 23:49:34,977] Trial 0 finished with value: 0.7355679702048418 and parameters: {'n_estimators': 168, 'max_depth': 19}. Best is trial 0 with value: 0.7355679702048418.
[I 2026-01-28 23:49:35,933] Trial 1 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 162, 'max_depth': 3}. Best is trial 1 with value: 0.7653631284916201.
[I 2026-01-28 23:49:36,307] Trial 2 finished with value: 0.7392923649906891 and parameters: {'n_estimators': 55, 'max_depth': 19}. Best is trial 1 with value: 0.7653631284916201.
[I 2026-01-28 23:49:37,069] Trial 3 finished with value: 0.7392923649906891 and parameters: {'n_estimators': 116, 'max_depth': 12}. Best is trial 1 with value: 0.7653631284916201.
[I 2026-01-28 23:49:38,621] Trial 4 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 196, 'max_depth': 4}. Best is trial 1 with value: 0.7653631

In [16]:
#print the best hyperparameters
print("best hyperparameters:", {study.best_trial.value})
print(f'Best hyperparameters: {study.best_trial.params}')

best hyperparameters: {0.7709497206703911}
Best hyperparameters: {'n_estimators': 157, 'max_depth': 3}


In [18]:
'''
Testing this best hyperparameters with random forest classifier
'''
from sklearn.metrics import accuracy_score

best_clf = RandomForestClassifier(**study.best_trial.params, random_state=42) #using best hyperparameters

best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print("Test set accuracy with best hyperparameters:", accuracy)


Test set accuracy with best hyperparameters: 0.7402597402597403


In [21]:
#Samplers in Optuna
#Random Sampler: Suggests hyperparameters randomly from the defined search space.
#TPE Sampler (Tree-structured Parzen Estimator): Uses a probabilistic model

study = optuna.create_study(sampler=optuna.samplers.RandomSampler())
study.optimize(objective, n_trials=50)

[I 2026-01-29 00:04:32,207] A new study created in memory with name: no-name-ec172a73-e494-49be-8d71-f1da334e2a71
[I 2026-01-29 00:04:32,828] Trial 0 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 99, 'max_depth': 4}. Best is trial 0 with value: 0.7635009310986964.
[I 2026-01-29 00:04:33,493] Trial 1 finished with value: 0.7392923649906891 and parameters: {'n_estimators': 101, 'max_depth': 15}. Best is trial 1 with value: 0.7392923649906891.
[I 2026-01-29 00:04:34,177] Trial 2 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 114, 'max_depth': 4}. Best is trial 1 with value: 0.7392923649906891.
[I 2026-01-29 00:04:35,275] Trial 3 finished with value: 0.750465549348231 and parameters: {'n_estimators': 177, 'max_depth': 8}. Best is trial 1 with value: 0.7392923649906891.
[I 2026-01-29 00:04:35,907] Trial 4 finished with value: 0.7523277467411545 and parameters: {'n_estimators': 101, 'max_depth': 7}. Best is trial 1 with value: 0.7392923649

In [20]:


# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7355679702048418
Best hyperparameters: {'n_estimators': 200, 'max_depth': 20}
